In [53]:
#pip install riotwatcher

In [1]:
import pandas as pd
import requests
from riotwatcher import LolWatcher, ApiError

In [2]:
with open('../.secret/Riot_API_key.txt','r') as f:
    Riot_API_key = f.readline()

In [3]:
lol_watcher = LolWatcher(Riot_API_key)

my_region = 'na1'

me = lol_watcher.summoner.by_name(my_region, 'Logan8586')
print(me)

# all objects are returned (by default) as a dict
# lets see if i got diamond yet (i probably didnt)
my_ranked_stats = lol_watcher.league.by_summoner(my_region, me['id'])
print(my_ranked_stats)

# First we get the latest version of the game from data dragon
versions = lol_watcher.data_dragon.versions_for_region(my_region)
champions_version = versions['n']['champion']

# Lets get some champions
current_champ_list = lol_watcher.data_dragon.champions(champions_version)
print(current_champ_list)

# For Riot's API, the 404 status code indicates that the requested data wasn't found and
# should be expected to occur in normal operation, as in the case of a an
# invalid summoner name, match ID, etc.
#
# The 429 status code indicates that the user has sent too many requests
# in a given amount of time ("rate limiting").

try:
    response = lol_watcher.summoner.by_name(my_region, 'this_is_probably_not_anyones_summoner_name')
except ApiError as err:
    if err.response.status_code == 429:
        print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
        print('this retry-after is handled by default by the RiotWatcher library')
        print('future requests wait until the retry-after time passes')
    elif err.response.status_code == 404:
        print('Summoner with that ridiculous name not found.')
    else:
        raise

{'id': '8kPSmXuAsHefD8d7_BNUvvu-YFfSqgiVdEQ-NvsAb3BGkd0', 'accountId': 'TMOX4oHMS40H-QCScrexrf_yaRUalsc1atT1elNv57x93w', 'puuid': 'XQepOwKh4QyRFj0O62e6y5TlB9x10xiPUuyTEFVwsKFWQxcv0vuVrdO5r90TpUIzbdhECRTVwxO9_Q', 'name': 'Logan8586', 'profileIconId': 26, 'revisionDate': 1632904670000, 'summonerLevel': 20}
[]
{'type': 'champion', 'format': 'standAloneComplex', 'version': '12.16.1', 'data': {'Aatrox': {'version': '12.16.1', 'id': 'Aatrox', 'key': '266', 'name': 'Aatrox', 'title': 'the Darkin Blade', 'blurb': 'Once honored defenders of Shurima against the Void, Aatrox and his brethren would eventually become an even greater threat to Runeterra, and were defeated only by cunning mortal sorcery. But after centuries of imprisonment, Aatrox was the first to find...', 'info': {'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4}, 'image': {'full': 'Aatrox.png', 'sprite': 'champion0.png', 'group': 'champion', 'x': 0, 'y': 0, 'w': 48, 'h': 48}, 'tags': ['Fighter', 'Tank'], 'partype': 'Blood We

In [4]:
current_champ_list_byId = [v['id'] for k,v in current_champ_list['data'].items()]
current_champ_list_byId

['Aatrox',
 'Ahri',
 'Akali',
 'Akshan',
 'Alistar',
 'Amumu',
 'Anivia',
 'Annie',
 'Aphelios',
 'Ashe',
 'AurelionSol',
 'Azir',
 'Bard',
 'Belveth',
 'Blitzcrank',
 'Brand',
 'Braum',
 'Caitlyn',
 'Camille',
 'Cassiopeia',
 'Chogath',
 'Corki',
 'Darius',
 'Diana',
 'Draven',
 'DrMundo',
 'Ekko',
 'Elise',
 'Evelynn',
 'Ezreal',
 'Fiddlesticks',
 'Fiora',
 'Fizz',
 'Galio',
 'Gangplank',
 'Garen',
 'Gnar',
 'Gragas',
 'Graves',
 'Gwen',
 'Hecarim',
 'Heimerdinger',
 'Illaoi',
 'Irelia',
 'Ivern',
 'Janna',
 'JarvanIV',
 'Jax',
 'Jayce',
 'Jhin',
 'Jinx',
 'Kaisa',
 'Kalista',
 'Karma',
 'Karthus',
 'Kassadin',
 'Katarina',
 'Kayle',
 'Kayn',
 'Kennen',
 'Khazix',
 'Kindred',
 'Kled',
 'KogMaw',
 'Leblanc',
 'LeeSin',
 'Leona',
 'Lillia',
 'Lissandra',
 'Lucian',
 'Lulu',
 'Lux',
 'Malphite',
 'Malzahar',
 'Maokai',
 'MasterYi',
 'MissFortune',
 'MonkeyKing',
 'Mordekaiser',
 'Morgana',
 'Nami',
 'Nasus',
 'Nautilus',
 'Neeko',
 'Nidalee',
 'Nilah',
 'Nocturne',
 'Nunu',
 'Olaf',
 'O

In [5]:
region = 'na1'
queue = 'RANKED_SOLO_5x5'
division_list = ['I','II','III','IV']

In [6]:
test = lol_watcher.league.challenger_by_queue(region= region, queue= queue)

In [77]:
#Apex Leagues
Leagues={}
Leagues['challenger']   = pd.DataFrame(lol_watcher.league.challenger_by_queue(region= region, queue= queue))
Leagues['grandmaster']  = pd.DataFrame(lol_watcher.league.grandmaster_by_queue(region= region, queue= queue))
Leagues['master']       = pd.DataFrame(lol_watcher.league.masters_by_queue(region= region, queue= queue))

#Diamond League is slightyly different. Even within divisions, multiple pages of that tier exists.
pg = 1
diamond_DF_list = []
while True:
    diamond_league_json = lol_watcher.league.entries(region= region, queue= queue, tier= 'DIAMOND', division= 'I', page= pg)
    if diamond_league_json:
        diamond_DF_list.append(pd.DataFrame(diamond_league_json))
    else:
        break    
    pg += 1

Leagues['diamond_I'] = pd.concat(diamond_DF_list)

#delete temporary list
del diamond_DF_list

In [79]:
for k,v in Leagues.items():
    print(f'{str.title(k)} League has {v.shape[0]} players')

Challenger League has 300 players
Grandmaster League has 700 players
Master League has 3663 players
Diamond_I League has 5743 players


In [76]:
me

{'id': '8kPSmXuAsHefD8d7_BNUvvu-YFfSqgiVdEQ-NvsAb3BGkd0',
 'accountId': 'TMOX4oHMS40H-QCScrexrf_yaRUalsc1atT1elNv57x93w',
 'puuid': 'XQepOwKh4QyRFj0O62e6y5TlB9x10xiPUuyTEFVwsKFWQxcv0vuVrdO5r90TpUIzbdhECRTVwxO9_Q',
 'name': 'Logan8586',
 'profileIconId': 26,
 'revisionDate': 1632904670000,
 'summonerLevel': 20}

# Challenger Tier
## Top 100 summoners in tier

In [122]:
top_100_challengerTier = pd.DataFrame(list(Leagues['challenger'].entries)).sort_values(by='leaguePoints',ascending=False).head(100).reset_index(drop=True)

summoner_info = {
    summoner_id:lol_watcher.summoner.by_id(region=region,encrypted_summoner_id=summoner_id) for summoner_id in list(top_100_challengerTier.summonerId)
}

top_100_challengerTier['puuid']         = top_100_challengerTier.summonerId.apply(lambda sum_id: summoner_info[sum_id]['puuid'])
top_100_challengerTier['accountId']     = top_100_challengerTier.summonerId.apply(lambda sum_id: summoner_info[sum_id]['accountId'])
top_100_challengerTier['profileIconId'] = top_100_challengerTier.summonerId.apply(lambda sum_id: summoner_info[sum_id]['profileIconId'])
top_100_challengerTier['summonerLevel'] = top_100_challengerTier.summonerId.apply(lambda sum_id: summoner_info[sum_id]['summonerLevel'])
top_100_challengerTier

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak,puuid,accountId,profileIconId,summonerLevel
0,M0p6yzSmGFyLB4BYtYzC_EexsSZFPr2tJXDFiX2qeZK1RYA,KatEvolved,1757,I,577,498,True,False,False,False,xJfznLAccbG197gLH2hvoFhJ0Kziq5tUXG8EHC8ZlVTRh3...,0Nyk49l2dDf-kE7ntljv6Y4cOFsDAYlVpdMzhH9T9tuSE1A,29,611
1,ohzlkyG9LMFRsn-7WLH35nzc084b4wckxQBpdad1oPC2SR...,MU APA,1752,I,300,220,True,False,False,True,RGh-AuTVPkJSs6WwO9XGKWq5p0Xws2qQ3Feu2ZU3x3KE_l...,Ev5l1w9ZYSCuW5vYt7fyu83WFRcN1yvR7fbuQa_u8agJsC...,3593,165
2,WwoB_gUKoA3kCQjQ61dqajF4iDeG5Pr2QTxArLDaL2bc,Ablazeolive,1669,I,260,185,True,False,False,False,KKXj8k9zF-kYRZMphQUjrDDTxyE69l61mcfHnnJL_0V6fs...,0H7G2NaSHZogSU_h48Zbgcx6NK-a6EJgQ9BbHcPTW1k,23,365
3,BwRKNFvxfJqpbQlEoAK0yC33AOfxN-UHg07hIF7SHTC6pSE,supp come top,1665,I,263,216,True,False,False,False,OdzZMyVGwLOZH0fJjVRLwMcKdlIXb-qcSDb1loflKASGlt...,fqGBM7bhjBCeKItmYM3olShxe027CrvS8spXM5U_1dB1aR8,1665,154
4,_2cIAZPFtUGYXAEPNtZrhUvg9m7dLs5BkzyZLMo-wM1cKC4,Cupic,1607,I,322,246,True,False,False,True,MEn8QXG_z84RdDYfqJ9_tpKuybhBJxyAnGzPEOC4P6qXgS...,fHbBCX4dURm3BgSyUxmQqIKmHtb0iyuGKwnpWQ3SLmvkVOA,1435,341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,lEvfte6xCrgrzYhklDZgvjV6bTNFLu49Tq9driBSGL6Myp...,SugiKim,1054,I,292,251,True,False,False,False,JmSxAiPoBl7w5P_IbEgV6VL6HsnaI65ZMN0ue9s1tAp5hD...,Y_d0HOLAsBPPxWnGTU_Y4cPL9UD2k9T8geuJ35BLK2aY0N...,20,138
96,7wOHZFPRdw_mlFe_VfvvLI6nZRLGWUhQwrZgDuABbiKjgGkE,Li Xiu Yue,1051,I,167,124,True,False,False,False,elSf0Adb1nwcB_vpJB0_m1fi0rlme_foXUTRXHJCe_k4B3...,ZH6YNlnEks2oALnWH8vxBuapHuxDVY2Y6BnaSWjXzcDNDy...,9,70
97,kkCvUPg_HaZgZTOUfXO5J-08Lw7ZJQU7JzRL-Hreq2xrYsA,JordanTheBoss,1051,I,786,731,False,False,False,False,T0CkSXKoDqe-xr4daHf8KzE5ACI2HBoBULJlZzmI6pccNA...,eFxXDO399_MEEn_9ceGGFCgM5zYEnZ7b452Laeg4vqXi9A,17,911
98,QWfPg-P-N-tsGi1BzOMKy7_qqCetrPPIhLLZg48mXfBfodA,Karen Osp fan,1050,I,295,244,False,False,False,False,4UKKw6o2n8pScw0wLBv4nKveAfuLz5UWCkD94wpoMDHR5P...,gY-gUnwxvJuOO9CoA4XtQqMPZ74TTHPeuCE1GsTtKebtWg,4895,266


## Get Last 100 Ranked Matches for each player

In [135]:
top_100_challengerTier_100matches  = set()

for i in range(100):
    top_100_challengerTier_100matches.update(
        lol_watcher.match.matchlist_by_puuid(
            region=region, 
            puuid = top_100_challengerTier.loc[i,'puuid'], 
            start = 0, 
            count = 100, 
            # queue: int = None, 
            type = 'ranked', 
            # start_time: int = None, 
            # end_time: int = None
        )
    )

In [141]:
lol_watcher.match.by_id(region=region,match_id= list(top_100_challengerTier_100matches)[0])

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_4423962964',
  'participants': ['EwC9RWntCwLxQ8YvWcGtNEKaH0w8VekD_xZQaPMJfW51iudiKOuWr2y8RCJI2bmbomqP103eZAaMvQ',
   '0WxP5bkOKQ84abcphnhcrHqq57zdREleBMYu0V_sspofaj134gopcOurUqZGKlxNqg--_If_QWFBtw',
   'JmSxAiPoBl7w5P_IbEgV6VL6HsnaI65ZMN0ue9s1tAp5hD9irn-gKr1bpB6chYgiINRmNXw-koEkVg',
   'EkOkBxqXafx8T5J0xjiPwPb4bVlzG2LWdnIlbctnhHowPWXGb7WBsJ3V2eZQ_Zg5nG27x0g5ghPKGg',
   'Uj12vnBJxBntCuCx70spftdWvfbHUcFntwCwaKZLhN-0s1-4GEL_qxYRh5v7CHeg5fwQXkuk70O2SQ',
   'S7JBLV51ww0cZwbXBNj5vbUNzSxGvrP9ybMD6QBWbfTalXJGvI2Y2_FDjugDHYjjq3njQNm4bl8qsA',
   'vOHkVSL-RTU3kW86EWed8McyhQp2tOwkpFhVeepBIAvJ_Am4PBN8e6G7QF56GWeX2bKiMs1FDg2W2Q',
   'Tc4CqLFmPDfwnh25oxvmKJu1Hx1I0wq5JKVVnRj-Rrq35FZcvUp24-l3Cjqe3VOdYcq8OvuqpdULuw',
   'STqLl-mcOY7EXV3zWsYLiSDrtWz0si4zTr2J4eyiWgeH6kqGquCspZsuAhcwOI5sDi_ZQ6h_oPAlug',
   'yJgQ0zRb2dYq9OEIjNwmmlFf8htTMF4sCs_uQRRSBR9iQRspdeZ5H402qe7LsksjHejedF2xu5QeTA']},
 'info': {'gameCreation': 1662149743515,
  'gameDuration': 916,
  '